In [1]:
import os
import numpy as np
# from torch.utils.data import DataLoader, Dataset

from preprocess import resize_input, train_test_split
from triplet_dataset import TripletDataset

import cv2
import torch


In [11]:
import importlib
from triplet_dataset import TripletDataset
import triplet_dataset

importlib.reload(triplet_dataset)
from triplet_dataset import TripletDataset

In [9]:
train_ds = torch.load("data/train_dataset.pt")
eval_ds = torch.load("data/eval_dataset.pt")

In [12]:
eval_ds = TripletDataset(eval_ds.data, eval_ds.labels)

In [13]:
# Test
(Xa, Xp, Xn), (ya, yp, yn) = eval_ds[0]


In [16]:
Xa.shape, ya, Xp.shape, yp, Xn.shape, yn

(torch.Size([3, 64, 64]),
 1254,
 torch.Size([3, 64, 64]),
 1254,
 torch.Size([3, 64, 64]),
 79)

In [17]:
torch.save(eval_ds, "data/eval_dataset.pt")

In [18]:
train_ds = TripletDataset(train_ds.data, train_ds.labels)

In [19]:
torch.save(train_ds, "data/train_dataset.pt")

In [20]:
len(eval_ds), len(train_ds)

(52270, 120809)

In [4]:
# from torch.utils.data import DataLoader

# dl = DataLoader(eval_dataset, 1, shuffle=True)
# sample = next(iter(dl))
# data, label = sample

# Xa, Xp, Xn = data
# ya, yp, yn = label

# Xa.shape

In [5]:
# torch.save(train_ds, "data/train_dataset.pt")
# torch.save(eval_ds, "data/eval_dataset.pt")


In [6]:
train_subjects = np.loadtxt("../train_subjects_mask.txt")

In [7]:
from PIL import Image
data_path = "../UERC"

In [8]:
ear_data = os.listdir(data_path)

ear_imgs = {}
for person in ear_data:
    if int(person) not in train_subjects:
        continue
    
    imgs = os.listdir("%s/%s" % (data_path, person))
    try:
        ear_imgs[person] = [
            cv2.cvtColor(
                np.asarray(Image.open(f"{data_path}/{person}/{img}")), cv2.COLOR_BGR2RGB
            )
            for img in imgs
        ]
    except Exception as e:
        print(e)

In [9]:
X_train, X_eval, y_train, y_eval = train_test_split(ear_imgs)

In [10]:
# Get train set size
len(X_train) + len(X_eval)

173079

In [11]:
len(X_train)

120809

In [12]:
X_train = resize_input(X_train, 64, mode="train")

In [15]:
import importlib
from triplet_dataset import TripletDataset
import triplet_dataset

importlib.reload(triplet_dataset)
from triplet_dataset import TripletDataset


In [16]:
train_dataset = TripletDataset(X_train, y_train)

In [17]:
(Xa, Xp, Xn), (ya, yp, yn) = train_dataset[0]

In [18]:
Xa.shape, Xp.shape, Xn.shape, ya, yp, yn

(torch.Size([3, 64, 64]),
 torch.Size([3, 64, 64]),
 torch.Size([3, 64, 64]),
 742,
 742,
 1182)

In [19]:
torch.save(train_dataset, "data/train_dataset.pt")

In [20]:
X_eval = resize_input(X_eval, 64, mode="test")

eval_dataset = TripletDataset(X_eval, y_eval)

In [21]:
from torch.utils.data import DataLoader

dl = DataLoader(eval_dataset, 1, shuffle=True)

In [22]:
sample = next(iter(dl))

In [23]:
data, label = sample

Xa, Xp, Xn = data
ya, yp, yn = label

Xa.shape

torch.Size([1, 3, 64, 64])

In [24]:
torch.save(eval_dataset, "data/eval_dataset.pt")
